In [1]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget.budget import locate_firsts
from voltagebudget.budget import locate_peaks
from voltagebudget.budget import filter_voltages
from fakespikes import util as fsutil

Loading BokehJS ...

# Define input

In [2]:
t = 0.4
stim_number = 40
stim_onset = 0.2
stim_offset = 0.350
stim_rate = 6
dt = 1e-5
seed_stim = 1
ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        n=stim_number,
        dt=dt,
        seed=None)

print(">>> {} spikes".format(ts_x.size))
print(">>> {} population rate".format(ts_x.size / (stim_offset - stim_onset)))

>>> 39 spikes
>>> 260.0 population rate


In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

- Shadow voltage response of an LIF neuron to the input

In [ ]:
#set mode
mode = 'regular'
params, w_in, bias_in, sigma = util.read_modes(mode)

In [ ]:
print(w_in, bias_in, sigma)

In [ ]:
# -
b_in = neurons.shadow_adex(10, t, ns_x, ts_x, f=0, A=0.05e-9, w_in=w_in, bias_in=bias_in, sigma=sigma, **params)
times = b_in['times']
v_in = b_in['V_m']

# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.2)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
combine = False
ns_p, ts_p = locate_peaks(b_in, stim_onset, stim_offset, combine=combine)
# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.1)
    if not combine:
        p.circle(x=ts_p[n], y=v_in[n, :].max(), color="purple")
    else:
        p.line(x=ts_p[0], y=np.linspace(v_in[:, :].min(), v_in[:, :].max(), 10), color="purple")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Adex response

In [ ]:
N = 20
A = .05e-9 *2
ns_y, ts_y, budget_y = neurons.adex(N, t, ns_x, ts_x, 
                                  w_in=w_in, 
                                  bias_in=bias_in, 
                                  sigma=0, 
                                  A=0, f=0, **params)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_y, ns_y, color="black")
p.x_range = Range1d(0.0, t)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(ts_y.size)

In [ ]:
ns_y

In [ ]:
times = budget_y['times']

p = figure(plot_width=400, plot_height=200)
for n in range(20):
    v = budget_y['V_m'][n, :]
    p.line(x=times, y=v, color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
# Extract budgets
combine = True
ns_first, ts_first = locate_firsts(ns_y, ts_y, combine=combine)
voltages_m = filter_voltages(
    budget_y,
    ns_first,
    ts_first,
    budget_delay=-8e-3,
    budget_width=1e-3,
    combine=combine)

times = budget_y['times']


p = figure(plot_width=400, plot_height=200)

for i, n in enumerate(set(ns_y)):
    v = voltages_m['V_m'][i, :]
    if not combine:
        times = voltages_m['times'][i, :]
    else:
        times = voltages_m['times']
    
    p.line(x=times, y=v, color="purple", alpha=0.8, line_width=3)
    
    v_full = budget_y['V_m'][n, :]
    times_full = budget_y['times']
    p.line(x=times_full, y=v_full, color="black", alpha=0.5, line_width=0.5)
    
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
# p.x_range = Range1d(0.63, 0.7)
p.y_range = Range1d(-66e-3, -48e-3)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget_y['times']
v = budget_y['V_m_thresh'][10, :]

p = figure(plot_width=400, plot_height=200)
p.line(x=times, y=v, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget_y['times']
v = budget_y['V_comp'][3, :]

p = figure(plot_width=400, plot_height=200)
p.line(x=times, y=v, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget_y['times']
v = budget_y['V_osc'][3, :]
Et = -52e-3
p = figure(title="{}".format(A), plot_width=400, plot_height=230)
p.line(x=times, y=v, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# p.y_range = Range1d(0.05, 0.08)
show(p)

In [ ]:
p = figure(plot_width=400, plot_height=200)
for i, n in enumerate(set(ns_y)):
    times = budget_y['times']
    v = budget_y['V_budget'][i] + budget_y['V_comp'][i, :] 
    p.line(x=times, y=v, color="black", alpha=0.2)

p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
np.min(voltages_m['V_free']) * 1e3, np.max(voltages_m['V_free']) * 1e3

In [ ]:
budget_y['V_budget'][i] - budget_y['V_comp'][i, :]

In [ ]:
np.mean(voltages_m['V_free']) * 1e3, np.std(voltages_m['V_free']) * 1e3